# Example to show word count using spark SQL

Get things set up.  On windows so I need these extra paths to be specified.

In [20]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import explode, split

Read in the data from a local source.  End up with a spark SQL data frame with one column (value) and one entry that is the text of the first chapter.

In [27]:
chap1 = spark.read.text("data/chap1.txt")
chap1.show()

+--------------------+
|               value|
+--------------------+
|chapter i  treats...|
+--------------------+



Now we'll use `split(str, regex, limit)`: Splits str around occurrences that match regex and returns an array with a length of at most limit.
Remember we have lazy eval though!

In [65]:
split(chap1.value, " ")

Column<'split(value,  , -1)'>

With the result of that, we'll use `explode()`: Separates the elements of array expr into multiple rows, or the elements of map expr into multiple rows and columns

In [66]:
explode(split(chap1.value, " "))

Column<'explode(split(value,  , -1))'>

As the column name isn't great, let's create an alias for the column name so it is easier to use.

In [72]:
explode(split(chap1.value, " ")).alias("word")

Column<'explode(split(value,  , -1)) AS `word`'>

Ok, so now we have a column object that we can select from our original `chap1` data frame.  We need to use the select method.
([syntax and more info](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.select.html))

In [74]:
col = explode(split(chap1.value, " ")).alias("word")
chap1.select(col)

DataFrame[word: string]

Use the `.show()` method to actually get the data back!

In [76]:
words = chap1.select(col)
words.show()

+-------------+
|         word|
+-------------+
|      chapter|
|            i|
|             |
|       treats|
|           of|
|          the|
|        place|
|        where|
|       oliver|
|        twist|
|          was|
|         born|
|          and|
|           of|
|          the|
|circumstances|
|    attending|
|          his|
|        birth|
|             |
+-------------+
only showing top 20 rows



Ok, that was a good check to make sure we had what we wanted.  Now we want to count the number of times each word occurs.  We'll use `groupBy()` and `count()` to do so.

In [77]:
words.groupBy("word").count()

DataFrame[word: string, count: bigint]

Let's use `.show()` to execute all the steps above and get something back!

In [79]:
counts = words.groupBy("word").count()
counts.show(30)

+-------------+-----+
|         word|count|
+-------------+-----+
|         some|    2|
|          few|    1|
|         hope|    1|
|    overseers|    2|
|   surrounded|    1|
|    biography|    1|
|  perspective|    1|
|circumstances|    1|
|  articulated|    1|
|        among|    1|
|          day|    1|
|         lips|    1|
|    appendage|    1|
|       raised|    2|
|      whether|    1|
|          did|    2|
|        space|    1|
|    existence|    1|
|          two|    1|
|     instance|    1|
|    buildings|    1|
|    strangers|    1|
|     occurred|    1|
|      inmates|    1|
|      backand|    1|
|       within|    1|
|       favour|    1|
|        could|    3|
|          him|    2|
|       badged|    1|
+-------------+-----+
only showing top 30 rows



Lastly, let's sort it and show some of the results.

In [84]:
counts.sort('count', ascending = False).show()

+-----+-----+
| word|count|
+-----+-----+
|  the|   75|
|     |   40|
|   of|   35|
|  and|   35|
|    a|   33|
|   to|   27|
|   in|   22|
|  was|   17|
|  her|   13|
|   it|   13|
| that|   12|
|  had|   12|
| have|   12|
|   by|   11|
|  his|   11|
| been|   11|
|  she|   11|
|   he|   11|
|which|   10|
|   on|   10|
+-----+-----+
only showing top 20 rows

